In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**LSTM**

In [11]:
import pandas as pd
data = pd.read_csv("data fix.csv")

In [12]:
data.dropna(axis=0, subset=['Article Content','Kategori'], inplace=True)
data.head()

,Nomor,Article Title,Article Link,Article Content,Kategori
0,1,"7 Hobi yang Bisa Datangkan Cuan di 2022, Ada H...",https://www.detik.com/edu/edutainment/d-588006...,Meski banyak orang menganggap hobi hanya sebat...,EDU
1,2,"Perbanyak Ahli Gunung Api, ITN Yogyakarta Buka...",https://www.detik.com/edu/perguruan-tinggi/d-5...,Institut Teknologi Nasional (ITN) Yogyakarta m...,EDU
2,3,"PTM Terbatas 2022 di Jakarta, Ini Aturan Kegia...",https://www.detik.com/edu/sekolah/d-5880075/pt...,Pemerintah Provinsi DKI Jakarta melalui Kepala...,EDU
3,4,Mengapa Perlu Menjaga Kelestarian Tanaman Baka...,https://www.detik.com/edu/detikpedia/d-5879161...,Tanaman bakau adalah salah satu ekosistem yang...,EDU
4,5,Cara Membuat Surat Lamaran Kerja yang Sempurna...,https://www.detik.com/edu/edutainment/d-587992...,Membuat Curriculum Vitae (CV) dan surat lamara...,EDU


In [13]:
data[130:140]

,Nomor,Article Title,Article Link,Article Content,Kategori
130,131,"Surat Az Zukhruf, Firman Allah SWT yang Mencer...",https://www.detik.com/edu/detikpedia/d-5883699...,"""Salah satu faedah surat Az Zukhruf adalah pen...",EDU
131,132,Doa Setelah Membaca Surat Al Waqiah dan Waktu ...,https://www.detik.com/edu/detikpedia/d-5883671...,Bacaan doa setelah membaca surat Al Waqiah ini...,EDU
132,133,"MUI Sudah 46 Tahun Berdiri, Seperti Apa Sejara...",https://www.detik.com/edu/detikpedia/d-5887336...,"Saat ini, Majelis Ulama Indonesia dipimpin ole...",EDU
133,134,Ortu di Jakarta yang Cemas PTM 100 % Diberi Ke...,https://www.detik.com/edu/sekolah/d-5887318/or...,Pembelajaran tatap muka (PTM) 100 persen sudah...,EDU
134,135,"Profil Kazakhstan, Negara Pecahan Uni Soviet d...",https://www.detik.com/edu/detikpedia/d-5887241...,Kazakhstan adalah negara pecahan Uni Soviet da...,EDU
135,136,Sultan Izinkan Sekolah-Kampus Gelar PTM 100 Pe...,https://www.detik.com/edu/perguruan-tinggi/d-5...,Gubernur Daerah Istimewa Yogyakarta Sri Sultan...,EDU
136,137,Ortu Was-was Anak Harus PTM 100 Persen? Psikol...,https://www.detik.com/edu/sekolah/d-5886908/or...,Kebijakan itu menuai pro dan kontra. Orang tua...,EDU
137,138,5 Contoh Teks Berita Dilengkapi Penjelasan dan...,https://www.detik.com/edu/detikpedia/d-5886893...,Kejadian atau peristiwa yang sedang dan telah ...,EDU
138,139,Ini Daftar Pekerjaan yang Bakal Digantikan Rob...,https://www.detik.com/edu/detikpedia/d-5886814...,Beberapa posisi pekerjaan saat ini sudah bisa ...,EDU
139,140,"Mengenal Serat, Bahan Baku Benang dan Kain",https://www.detik.com/edu/detikpedia/d-5886805...,"Mengutip dari Sumber Belajar Kemdikbud, sifat ...",EDU


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X_train,X_test, y_train, y_test = train_test_split(data['Article Content'],data['Kategori'],train_size=0.8,random_state=42)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [15]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='#$$%&()*+,-./:;<=>?@[\]^_{|}~', lower=True)
tokenizer.fit_on_texts(data['Article Content'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = tokenizer.texts_to_sequences(data['Article Content'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Found 43383 unique tokens.
Shape of data tensor: (2493, 250)


In [16]:
y = pd.get_dummies(data['Kategori']).values
print('Shape of label tensor:', y.shape)

Shape of label tensor: (2493, 6)


In [17]:
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.10, random_state = 42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(2243, 250) (2243, 6)
(250, 250) (250, 6)


In [18]:
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 30
batch_size = 64

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Epoch 1/30
32/32 [==============================] - 61s 2s/step - loss: 1.7168 - accuracy: 0.3722 - val_loss: 1.6300 - val_accuracy: 0.4267
Epoch 2/30
32/32 [==============================] - 57s 2s/step - loss: 1.3490 - accuracy: 0.5273 - val_loss: 1.3209 - val_accuracy: 0.4667
Epoch 3/30
32/32 [==============================] - 69s 2s/step - loss: 0.9578 - accuracy: 0.6477 - val_loss: 1.0271 - val_accuracy: 0.5733
Epoch 4/30
32/32 [==============================] - 73s 2s/step - loss: 0.5424 - accuracy: 0.8067 - val_loss: 0.8438 - val_accuracy: 0.6578
Epoch 5/30
32/32 [==============================] - 67s 2s/step - loss: 0.2624 - accuracy: 0.9108 - val_loss: 0.6775 - val_accuracy: 0.7644
Epoch 6/30
32/32 [==============================] - 60s 2s/step - loss: 0.1199 - accuracy: 0.9727 - val_loss: 0.6670 - val_accuracy: 0.7956
Epoch 7/30
32/32 [==============================] - 57s 2s/step - loss: 0.0646 - accuracy: 0.9891 - val_loss: 0.6803 - val_accuracy: 0.8133
Epoch 8/30
32/32 [==

In [19]:
y_pred = model.predict(X_test)
y_pred.shape

(250, 6)

**RNN**

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [21]:
dataset_train = pd.read_csv("data fix.csv")
training_set = dataset_train.iloc[:, 1:2].values
dataset_train.head()

,Nomor,Article Title,Article Link,Article Content,Kategori
0,1,"7 Hobi yang Bisa Datangkan Cuan di 2022, Ada H...",https://www.detik.com/edu/edutainment/d-588006...,Meski banyak orang menganggap hobi hanya sebat...,EDU
1,2,"Perbanyak Ahli Gunung Api, ITN Yogyakarta Buka...",https://www.detik.com/edu/perguruan-tinggi/d-5...,Institut Teknologi Nasional (ITN) Yogyakarta m...,EDU
2,3,"PTM Terbatas 2022 di Jakarta, Ini Aturan Kegia...",https://www.detik.com/edu/sekolah/d-5880075/pt...,Pemerintah Provinsi DKI Jakarta melalui Kepala...,EDU
3,4,Mengapa Perlu Menjaga Kelestarian Tanaman Baka...,https://www.detik.com/edu/detikpedia/d-5879161...,Tanaman bakau adalah salah satu ekosistem yang...,EDU
4,5,Cara Membuat Surat Lamaran Kerja yang Sempurna...,https://www.detik.com/edu/edutainment/d-587992...,Membuat Curriculum Vitae (CV) dan surat lamara...,EDU


In [22]:
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, SimpleRNN, Dense

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(SimpleRNN(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 30
batch_size = 64

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Epoch 1/30
32/32 [==============================] - 21s 491ms/step - loss: 1.7658 - accuracy: 0.2413 - val_loss: 1.6459 - val_accuracy: 0.3067
Epoch 2/30
32/32 [==============================] - 18s 566ms/step - loss: 1.5919 - accuracy: 0.3627 - val_loss: 1.6174 - val_accuracy: 0.3600
Epoch 3/30
32/32 [==============================] - 17s 533ms/step - loss: 1.4827 - accuracy: 0.4014 - val_loss: 1.4766 - val_accuracy: 0.3689
Epoch 4/30
32/32 [==============================] - 19s 585ms/step - loss: 1.4020 - accuracy: 0.4356 - val_loss: 1.4527 - val_accuracy: 0.4267
Epoch 5/30
32/32 [==============================] - 22s 696ms/step - loss: 1.3882 - accuracy: 0.4495 - val_loss: 1.5013 - val_accuracy: 0.3778
Epoch 6/30
32/32 [==============================] - 19s 606ms/step - loss: 1.3665 - accuracy: 0.4648 - val_loss: 1.4532 - val_accuracy: 0.4000
Epoch 7/30
32/32 [==============================] - 17s 519ms/step - loss: 1.3440 - accuracy: 0.4628 - val_loss: 1.4410 - val_accuracy: 0.4089

**CNN**

In [23]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Activation,  Dropout 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [24]:
data = pd.read_csv("data fix.csv")
data.head()

,Nomor,Article Title,Article Link,Article Content,Kategori
0,1,"7 Hobi yang Bisa Datangkan Cuan di 2022, Ada H...",https://www.detik.com/edu/edutainment/d-588006...,Meski banyak orang menganggap hobi hanya sebat...,EDU
1,2,"Perbanyak Ahli Gunung Api, ITN Yogyakarta Buka...",https://www.detik.com/edu/perguruan-tinggi/d-5...,Institut Teknologi Nasional (ITN) Yogyakarta m...,EDU
2,3,"PTM Terbatas 2022 di Jakarta, Ini Aturan Kegia...",https://www.detik.com/edu/sekolah/d-5880075/pt...,Pemerintah Provinsi DKI Jakarta melalui Kepala...,EDU
3,4,Mengapa Perlu Menjaga Kelestarian Tanaman Baka...,https://www.detik.com/edu/detikpedia/d-5879161...,Tanaman bakau adalah salah satu ekosistem yang...,EDU
4,5,Cara Membuat Surat Lamaran Kerja yang Sempurna...,https://www.detik.com/edu/edutainment/d-587992...,Membuat Curriculum Vitae (CV) dan surat lamara...,EDU


In [25]:
text = data["Article Content"].tolist()
print(len(text))

2493


In [26]:
#Case Folding
import pandas
data['Article Content'] = data['Article Content'].str.lower()

print('Case Folding Result : \n')
print(data['Article Content'].head(5))
print('\n\n\n')

Case Folding Result : 

0    meski banyak orang menganggap hobi hanya sebat...
1    institut teknologi nasional (itn) yogyakarta m...
2    pemerintah provinsi dki jakarta melalui kepala...
3    tanaman bakau adalah salah satu ekosistem yang...
4    membuat curriculum vitae (cv) dan surat lamara...
Name: Article Content, dtype: object






In [27]:
token = Tokenizer()
token.fit_on_texts(text)

In [28]:
token.index_word

{1: 'yang',
 2: 'di',
 3: 'dan',
 4: 'ini',
 5: 'dengan',
 6: 'dari',
 7: 'untuk',
 8: 'itu',
 9: 'dalam',
 10: 'pada',
 11: 'juga',
 12: 'tidak',
 13: '1',
 14: 'akan',
 15: 'ada',
 16: 'ke',
 17: 'tahun',
 18: '2022',
 19: 'tersebut',
 20: 'sudah',
 21: 'bisa',
 22: 'atau',
 23: 'saat',
 24: 'adalah',
 25: 'orang',
 26: 'oleh',
 27: 'indonesia',
 28: 'menjadi',
 29: 'karena',
 30: '2',
 31: 'dia',
 32: 'lebih',
 33: 'rp',
 34: 'mereka',
 35: 'video',
 36: 'sebagai',
 37: '2021',
 38: 'saya',
 39: 'satu',
 40: '3',
 41: 'kita',
 42: 'dapat',
 43: 'kata',
 44: 'seperti',
 45: 'baru',
 46: 'hal',
 47: 'masih',
 48: 'hari',
 49: 'kami',
 50: 'telah',
 51: 'anak',
 52: 'harus',
 53: 'hingga',
 54: 'banyak',
 55: 'kepada',
 56: 'mobil',
 57: 'jadi',
 58: 'memiliki',
 59: 'lalu',
 60: '4',
 61: 'sekolah',
 62: 'tak',
 63: 'ia',
 64: 'secara',
 65: 'salah',
 66: '5',
 67: 'namun',
 68: 'melakukan',
 69: 'hanya',
 70: 'pun',
 71: '000',
 72: 'bagi',
 73: 'lagi',
 74: 'setelah',
 75: 'jika',
 

In [29]:
vocal = len(token.index_word)+1
vocal

41295

In [30]:
x =['aku lagi galau nih']
token.texts_to_sequences(x)

[[117, 73, 15671, 1723]]

In [31]:
encode_text = token.texts_to_sequences(text)
print(encode_text)

In [32]:
max_kata = 100
X = pad_sequences(encode_text, maxlen = max_kata, padding="post")
X

array([[  159,    36,    35, ...,   120,  2440,  1331],
       [  103,    59,   768, ...,   138,    10,    37],
       [  240,     5,   756, ...,     5,    90,   106],
       ...,
       [10098,  1088,   489, ...,     3,   577,  2042],
       [   88,   243,     4, ...,     0,     0,     0],
       [    4,    58,  3664, ...,    17,    37,    59]])

In [33]:
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.10, random_state = 42)


In [34]:
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [35]:
#Model CNN
vec_size=300
model = Sequential()
model.add(Embedding(vocal, vec_size, input_length=max_kata))
model.add(Conv1D(64,8, activation="relu"))
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))

model.add(Dense(8, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(16, activation="relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation="softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 300)          12388500  
                                                                 
 conv1d (Conv1D)             (None, 93, 64)            153664    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 46, 64)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 46, 64)            0         
                                                                 
 dense_2 (Dense)             (None, 46, 8)             520       
                                                                 
 dropout_1 (Dropout)         (None, 46, 8)             0         
                                                      

In [36]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])